

## ✅  查看版本信息

In [ ]:
# check installed version
import pycaret
pycaret.__version__



## ✅  数据预处理



## 01加载数据
训练集\测试集\生成数据集

In [ ]:
import os
os.chdir('/program/IAD_ML')
from data_precess.get_example_datas import load_and_prepare_data
from data_precess.get_acverge_prediction_score import calculate_and_display_average_prediction_score

# 设置基础路径
base_path = '/program/IAD_ML'

# 设置数据集类型
# 01 tightening_data
# data_type = 'tightening_data'

# 02 validation_data
data_type = 'chinatown_data'
# data_type = 'sony_ai_robot_data'
# data_type = 'gunpoint_oldversusyoung_data'


# # 01 tightening_data
# data_folder = os.path.join(base_path, 'data_sets_12_24', f'{data_type}', 'exp_feature_data', 'timegan')

# # 02 sony_ai_robot_data
data_folder = os.path.join(base_path, 'data_sets_12_24', 'validation_data', f'{data_type}', 'exp_feature_data')

# Construct file paths
file_path_train_data = os.path.join(data_folder, 'train_data.csv')
file_path_test_data = os.path.join(data_folder, 'test_data.csv')
file_path_syn_data = os.path.join(data_folder, 'syn_data.csv')

try:
    # Load and prepare data
    train_data, test_data, syn_data = load_and_prepare_data(file_path_train_data, file_path_test_data, file_path_syn_data)
    
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
syn_data.head()

## 02去除不重要的特征
特征1、Maximum特征2、Minimum特征3、Mean特征4、Standard Deviation特征5、Root Mean Square特征6、Skewness特征7、Kurtosis特征8、Peak Value特征9、Peak Value Index特征10、Shape Index特征11、Absolute Mean特征12、Sample Variance特征13、Root Amplitude特征14、Impulse Index特征15、Margin Index

In [ ]:
# # 去除特征Impulse Index
# train_data.drop('Impulse Index', axis=1, inplace=True)
# test_data.drop('Impulse Index', axis=1, inplace=True)
# syn_data.drop('Impulse Index', axis=1, inplace=True)

In [ ]:
# # 预览剔除不重要特征后的数据集
# train_data.head()

In [ ]:
# # 预览剔除不重要特征后的数据集
# test_data.head()

In [ ]:
# # 预览剔除不重要特征后的数据集
# syn_data.head()



## ✅  设置实验数据集合
测试集数据保持不变,将不同比例的生成数据加入测试集(0.00/0.25/0.50/0.75/1.00)*生成数据

In [ ]:
import pandas as pd

# 计算要合并的合成样本数
ratio = 0.00
syn_plus_samples = len(syn_data)
num_syn_samples = int(ratio * syn_plus_samples)

# 选择前 num_syn_samples 个合成样本
synthetic_samples = syn_data.iloc[:num_syn_samples]
new_train_data = pd.concat([train_data, synthetic_samples], axis=0).reset_index(drop=True)
new_train_data.head()

In [ ]:
# 设置保存路径（训练集、测试集、模型对比）
data_folder_setup_info = os.path.join(base_path,
                                          'experimental_results',
                                          'validation_data',
                                          f'{data_type}',
                                          f'experiment_{ratio:.2f}',
                                          'setup_info_and_results')

In [ ]:
# 保存为CSV文件（第一次运行保存即可，后续不用保存，一般第一次会运行xgboost）
save_path_new_train_data = os.path.join(data_folder_setup_info, 'new_train_data.csv')
new_train_data.to_csv(save_path_new_train_data, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_new_train_data):
    print(f"CSV file saved successfully at: {save_path_new_train_data}")
else:
    print(f"Failed to save CSV file at: {save_path_new_train_data}")

In [ ]:
# copy data and drop Class variable
test_data_copy = test_data.copy()
test_data_copy.head()

In [ ]:
# 获取 new_train_data 的行数
new_train_data_length = new_train_data.shape[0]

# 重置 test_data 的索引，并加上 train_data 的行数
new_test_data = test_data_copy.reset_index(drop=True)
new_test_data.index = new_test_data.index + new_train_data_length
new_test_data.head()

In [ ]:
# 保存为CSV文件（第一次运行保存即可，后续不用保存，一般第一次会运行xgboost）
save_path_new_test_data = os.path.join(data_folder_setup_info, 'new_test_data.csv')
new_test_data.to_csv(save_path_new_test_data, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_new_test_data):
    print(f"CSV file saved successfully at: {save_path_new_test_data}")
else:
    print(f"Failed to save CSV file at: {save_path_new_test_data}")

## ✅ 设置并启动模型

In [ ]:
# import pycaret classification and init setup
from pycaret.classification import *

# 设置折叠策略
fold_strategy = 'stratifiedkfold'
# 设置折叠数
num_folds = 10
# 是否在每个折之前洗牌数据
fold_shuffle = True
# 是否对数值型特征进行标准化
normalize_data = True
# 设置 session_id 以确保可重复性
session_id = 123

# 初始化 PyCaret
s = setup(
    data=new_train_data,
    test_data=new_test_data,
    target='labels',
    fold_strategy=fold_strategy,
    fold=num_folds,
    fold_shuffle=fold_shuffle,
    normalize=normalize_data,
    session_id=session_id
)


In [ ]:
# help(setup)

## ✅ 比较模型

In [ ]:
# compare baseline models
best = compare_models()

In [ ]:
# help(compare_models)

In [ ]:
# 获取比较结果的数据框（第一次运行保存即可，后续不用保存，一般第一次会运行xgboost）
compare_results = pull()

# 保存为CSV文件
save_path_compare = os.path.join(data_folder_setup_info, 'compare_results.csv')
compare_results.to_csv(save_path_compare, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_compare):
    print(f"CSV file saved successfully at: {save_path_compare}")
else:
    print(f"Failed to save CSV file at: {save_path_compare}")

## ✅ 分析指定模型

## 01创建和优化模型

In [ ]:
# 指定模型
model_name = 'xgboost'

# 设置保存路径
data_folder_model = os.path.join(base_path,
                                          'experimental_results',
                                          'validation_data',
                                          f'{data_type}',
                                          f'experiment_{ratio:.2f}',
                                          f'model_{model_name}')

## 01.1创建模型

In [ ]:
# train lr and return train score as well alongwith CV
create_model = create_model(model_name)

# 获取比较结果的数据框
create_model_results = pull()

In [ ]:
# 保存为CSV文件
save_path_create = os.path.join(data_folder_model, 'create_model_results.csv')
create_model_results.to_csv(save_path_create, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_create):
    print(f"CSV file saved successfully at: {save_path_create}")
else:
    print(f"Failed to save CSV file at: {save_path_create}")

## 01.2优化模型

In [ ]:
# 初始化调参
tuned_model = tune_model(create_model, search_library='optuna')

# 获取调优模型性能指标
tuned_model_results = pull()

In [ ]:
# 获取评估指标的列名
metric_column_name = 'Accuracy'  # 选择评价指标，可以根据实际需求更改

# 设置最大循环次数
max_iterations = 10
iteration_count = 0

# 获取初始模型性能指标
initial_metric = create_model_results[metric_column_name].iloc[-2]

# 获取调优模型性能指标
tuned_metric = tuned_model_results[metric_column_name].iloc[-2]

# 比较初始模型和调优模型性能
while tuned_metric < initial_metric and iteration_count < max_iterations:
    # 继续优化
    tuned_model = tune_model(create_model, search_library='optuna')
    
    # 获取调优模型性能指标
    tuned_model_results = pull()
    tuned_metric = tuned_model_results[metric_column_name].iloc[-2]  # 选择倒数第二行的值
    
    # 增加循环计数器
    iteration_count += 1

# 在循环结束后选择最好的模型
# better_model = tuned_model if tuned_metric > initial_metric else create_model
better_model = tuned_model if tuned_metric >= initial_metric else create_model

In [ ]:
# help(tuned_model)

In [ ]:
# 保存为CSV文件
save_path_tuned = os.path.join(data_folder_model, 'tuned_model_results.csv')
tuned_model_results.to_csv(save_path_tuned, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_tuned):
    print(f"CSV file saved successfully at: {save_path_tuned}")
else:
    print(f"Failed to save CSV file at: {save_path_tuned}")

## 02模型评估

In [ ]:
# 自定义保存基础路径
base_path_images = r'C:\Users\AdmaZ\Desktop\experimental_results'

# 设置保存路径
images_save_path = os.path.join(base_path_images,
                                f'{data_type}',
                                f'experiment_{ratio:.2f}',
                                f'model_{model_name}')

## 02.1解释模型

In [ ]:
# interpret summary model（不支持:lda，lr，gbc，nb，knn，svm）
interpret_model(better_model, plot = 'summary', save=images_save_path)

In [ ]:
# interpret correlation model（不支持:lda，lr，gbc，nb，knn，svm）
interpret_model(better_model, plot = 'correlation', save=images_save_path)

In [ ]:
# # interpret summary model
# interpret_model(tuned_model, plot = 'reason')

In [ ]:
# # interpret summary model
# interpret_model(tuned_model, plot = 'pdp')

In [ ]:
# # interpret summary model
# interpret_model(tuned_model, plot = 'msa')

In [ ]:
# # interpret summary model
# interpret_model(tuned_model, plot = 'pfi')

In [ ]:
# help(interpret_model)

## 02.2评估模型

In [ ]:
evaluate_model(tuned_model)

## 02.3绘图模型

In [ ]:
# 01 Precision Recall
file_path_pr = plot_model(better_model, plot='pr', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_pr):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_pr}")
else:
    print("图片保存失败。")

In [ ]:
# 02 Manifold Learning
file_path_manifold = plot_model(better_model, plot='manifold', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_manifold):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_manifold}")
else:
    print("图片保存失败。")

In [ ]:
# 03 Feature Importance plot（不支持：nb，knn，
file_path_feature_all = plot_model(better_model, plot='feature_all', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_feature_all):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_feature_all}")
else:
    print("图片保存失败。")

In [ ]:
# 04 KS Statistic Plot（不支持：svm，
file_path_ks = plot_model(better_model, plot='ks', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_ks):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_ks}")
else:
    print("图片保存失败。")

In [ ]:
# 05 Hyperparameters
file_path_parameter = plot_model(better_model, plot='parameter', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_parameter):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_parameter}")
else:
    print("图片保存失败。")

In [ ]:
# 06 Prediction Error
file_path_error = plot_model(better_model, plot='error', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_error):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_error}")
else:
    print("图片保存失败。")

In [ ]:
# 07 Calibration Curve（不支持：svm，
file_path_calibration = plot_model(better_model, plot='calibration', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_calibration):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_calibration}")
else:
    print("图片保存失败。")

In [ ]:
# 08 Decision Boundary
file_path_boundary = plot_model(better_model, plot='boundary', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_boundary):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_boundary}")
else:
    print("图片保存失败。")

In [ ]:
# 09 AUC（不支持：svm，
file_path_auc = plot_model(better_model, plot='auc', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_auc):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_auc}")
else:
    print("图片保存失败。")

In [ ]:
# 10 Class Report
file_path_class_report = plot_model(better_model, plot='class_report', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_class_report):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_class_report}")
else:
    print("图片保存失败。")

In [ ]:
# 11 Validation Curve（不支持：lda，
file_path_vc = plot_model(better_model, plot='vc', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_vc):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_vc}")
else:
    print("图片保存失败。")

In [ ]:
# 12 Lift Chart（不支持：svm，
file_path_lift = plot_model(better_model, plot='lift', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_lift):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_lift}")
else:
    print("图片保存失败。")

In [ ]:
# 13 Confusion Matrix
file_path_confusion_matrix = plot_model(better_model, plot='confusion_matrix', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_confusion_matrix):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_confusion_matrix}")
else:
    print("图片保存失败。")

In [ ]:
# 14 Feature Selection（不支持：nb，knn，
file_path_rfe = plot_model(better_model, plot='rfe', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_rfe):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_rfe}")
else:
    print("图片保存失败。")

In [ ]:
# 15 Dimensions
file_path_dimension = plot_model(better_model, plot='dimension', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_dimension):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_dimension}")
else:
    print("图片保存失败。")

In [ ]:
# 16 Gain Chart（不支持：svm，
file_path_gain = plot_model(better_model, plot='gain', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_gain):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_gain}")
else:
    print("图片保存失败。")

In [ ]:
# 17 Threshold
file_path_threshold = plot_model(better_model, plot='threshold', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_threshold):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_threshold}")
else:
    print("图片保存失败。")

In [ ]:
# 18 Learning Curve
file_path_learning = plot_model(better_model, plot='learning', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_learning):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_learning}")
else:
    print("图片保存失败。")

In [ ]:
# 19 Feature Importance（不支持：nb，knn，
file_path_feature = plot_model(better_model, plot='feature', save=images_save_path)

# 检查图片是否存在
if os.path.exists(file_path_feature):
    print("图片保存成功！")
    print(f"生成的图片文件名：{file_path_feature}")
else:
    print("图片保存失败。")

In [ ]:
# # 20 Decision Tree（不支持：xgboost，dt，lightgbm，rf，lda，gbc，knn，svm)
# file_path_tree = plot_model(better_model, plot='tree', save=images_save_path)

# # 检查图片是否存在
# if os.path.exists(file_path_tree):
#     print("图片保存成功！")
#     print(f"生成的图片文件名：{file_path_tree}")
# else:
#     print("图片保存失败。")

In [ ]:
# help(plot_model)

## 03获取正样本和负样本的平均预测概率

## 03.1再次划分测试集

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

# 随机打乱 test_data
shuffled_new_test_data = shuffle(new_test_data, random_state=42)

# 将 test_data 分成四份
num_splits = 5
splitted_data = np.array_split(shuffled_new_test_data, num_splits)


# 打印每份的大小
for i, split in enumerate(splitted_data):
    print(f"Split {i + 1}: {len(split)} rows")

# 可以通过索引访问每份数据
test_data_split_1 = splitted_data[0]
test_data_split_2 = splitted_data[1]
test_data_split_3 = splitted_data[2]
test_data_split_4 = splitted_data[3]
test_data_split_5 = splitted_data[4]

## 03.2实验验证

## 实验一

In [ ]:
# predict on test set
holdout_pred_1 = predict_model(better_model, data=test_data_split_1)

In [ ]:
# 获取比较结果的数据框
holdout_pred_1_results = pull()

# 保存为CSV文件
save_path_holdout_pred_1_results = os.path.join(data_folder_model, 'holdout_pred_1_results.csv')
holdout_pred_1_results.to_csv(save_path_holdout_pred_1_results, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_holdout_pred_1_results):
    print(f"CSV file saved successfully at: {save_path_holdout_pred_1_results}")
else:
    print(f"Failed to save CSV file at: {save_path_holdout_pred_1_results}")

In [ ]:
# show predictions df
holdout_pred_1.head()

In [ ]:
# 保存为CSV文件
save_path_holdout_pred_1 = os.path.join(data_folder_model, 'holdout_pred_1.csv')
holdout_pred_1.to_csv(save_path_holdout_pred_1, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_holdout_pred_1):
    print(f"CSV file saved successfully at: {save_path_holdout_pred_1}")
else:
    print(f"Failed to save CSV file at: {save_path_holdout_pred_1}")
    

In [ ]:
# 计算平均概率：（不支持：svm
# 正样本
apsl_nor_exp_1 = calculate_and_display_average_prediction_score(holdout_pred_1, 0)
print(f"{apsl_nor_exp_1}")

# 负样本
apsl_abn_exp_1 = calculate_and_display_average_prediction_score(holdout_pred_1, 1)
print(f"{apsl_abn_exp_1}")

## 实验二

In [ ]:
# predict on test set
holdout_pred_2 = predict_model(better_model, data=test_data_split_2)

In [ ]:
# 获取比较结果的数据框
holdout_pred_2_results = pull()

# 保存为CSV文件
save_path_holdout_pred_2_results = os.path.join(data_folder_model, 'holdout_pred_2_results.csv')
holdout_pred_2_results.to_csv(save_path_holdout_pred_2_results, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_holdout_pred_2_results):
    print(f"CSV file saved successfully at: {save_path_holdout_pred_2_results}")
else:
    print(f"Failed to save CSV file at: {save_path_holdout_pred_2_results}")

In [ ]:
# show predictions df
holdout_pred_2.head()

In [ ]:
# 保存为CSV文件
save_path_holdout_pred_2 = os.path.join(data_folder_model, 'holdout_pred_2.csv')
holdout_pred_2.to_csv(save_path_holdout_pred_2, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_holdout_pred_2):
    print(f"CSV file saved successfully at: {save_path_holdout_pred_2}")
else:
    print(f"Failed to save CSV file at: {save_path_holdout_pred_2}")
    

In [ ]:
# 计算平均概率：（不支持：svm
# 正样本
apsl_nor_exp_2 = calculate_and_display_average_prediction_score(holdout_pred_2, 0)

# 负样本
apsl_abn_exp_2 = calculate_and_display_average_prediction_score(holdout_pred_2, 1)

## 实验三

In [ ]:
# predict on test set
holdout_pred_3 = predict_model(better_model, data=test_data_split_3)

In [ ]:
# 获取比较结果的数据框
holdout_pred_3_results = pull()

# 保存为CSV文件
save_path_holdout_pred_3_results = os.path.join(data_folder_model, 'holdout_pred_3_results.csv')
holdout_pred_3_results.to_csv(save_path_holdout_pred_3_results, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_holdout_pred_3_results):
    print(f"CSV file saved successfully at: {save_path_holdout_pred_3_results}")
else:
    print(f"Failed to save CSV file at: {save_path_holdout_pred_3_results}")

In [ ]:
# show predictions df
holdout_pred_3.head()

In [ ]:
# 保存为CSV文件
save_path_holdout_pred_3 = os.path.join(data_folder_model, 'holdout_pred_3.csv')
holdout_pred_3.to_csv(save_path_holdout_pred_3, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_holdout_pred_3):
    print(f"CSV file saved successfully at: {save_path_holdout_pred_3}")
else:
    print(f"Failed to save CSV file at: {save_path_holdout_pred_3}")
    

In [ ]:
# 计算平均概率：（不支持：svm
# 正样本
apsl_nor_exp_3 = calculate_and_display_average_prediction_score(holdout_pred_3, 0)

# 负样本
apsl_abn_exp_3 = calculate_and_display_average_prediction_score(holdout_pred_3, 1)

## 实验四

In [ ]:
# predict on test set
holdout_pred_4 = predict_model(better_model, data=test_data_split_4)

In [ ]:
# 获取比较结果的数据框
holdout_pred_4_results = pull()

# 保存为CSV文件
save_path_holdout_pred_4_results = os.path.join(data_folder_model, 'holdout_pred_4_results.csv')
holdout_pred_4_results.to_csv(save_path_holdout_pred_4_results, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_holdout_pred_4_results):
    print(f"CSV file saved successfully at: {save_path_holdout_pred_4_results}")
else:
    print(f"Failed to save CSV file at: {save_path_holdout_pred_4_results}")

In [ ]:
# show predictions df
holdout_pred_4.head()

In [ ]:
# 保存为CSV文件
save_path_holdout_pred_4 = os.path.join(data_folder_model, 'holdout_pred_4.csv')
holdout_pred_4.to_csv(save_path_holdout_pred_4, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_holdout_pred_4):
    print(f"CSV file saved successfully at: {save_path_holdout_pred_4}")
else:
    print(f"Failed to save CSV file at: {save_path_holdout_pred_4}")
    

In [ ]:
# 计算平均概率：（不支持：svm
# 正样本
apsl_nor_exp_4 = calculate_and_display_average_prediction_score(holdout_pred_4, 0)

# 负样本
apsl_abn_exp_4 = calculate_and_display_average_prediction_score(holdout_pred_4, 1)

## 实验五

In [ ]:
# predict on test set
holdout_pred_5 = predict_model(better_model, data=test_data_split_5)

In [ ]:
# 获取比较结果的数据框
holdout_pred_5_results = pull()

# 保存为CSV文件
save_path_holdout_pred_5_results = os.path.join(data_folder_model, 'holdout_pred_5_results.csv')
holdout_pred_5_results.to_csv(save_path_holdout_pred_5_results, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_holdout_pred_5_results):
    print(f"CSV file saved successfully at: {save_path_holdout_pred_5_results}")
else:
    print(f"Failed to save CSV file at: {save_path_holdout_pred_5_results}")

In [ ]:
# show predictions df
holdout_pred_5.head()

In [ ]:
# 保存为CSV文件
save_path_holdout_pred_5 = os.path.join(data_folder_model, 'holdout_pred_5.csv')
holdout_pred_5.to_csv(save_path_holdout_pred_5, index=False)

# 检查文件是否成功保存
if os.path.exists(save_path_holdout_pred_5):
    print(f"CSV file saved successfully at: {save_path_holdout_pred_5}")
else:
    print(f"Failed to save CSV file at: {save_path_holdout_pred_5}")

In [ ]:
# 计算平均概率：（不支持：svm
# 正样本
apsl_nor_exp_5 = calculate_and_display_average_prediction_score(holdout_pred_5, 0)

# 负样本
apsl_abn_exp_5 = calculate_and_display_average_prediction_score(holdout_pred_5, 1)

## 03.3整合所有平均预测概率、保存

In [ ]:
# 计算平均概率（不支持：svm
import pandas as pd


def calculate_mean(exp_list, exp_name):
    mean_exp_list = sum(exp_list) / len(exp_list)
    result = {exp_name: exp_list + [mean_exp_list]}
    return result


apsl_nor_list = [apsl_nor_exp_1, apsl_nor_exp_2, apsl_nor_exp_3, apsl_nor_exp_4, apsl_nor_exp_5]
apsl_abn_list = [apsl_abn_exp_1, apsl_abn_exp_2, apsl_abn_exp_3, apsl_abn_exp_4, apsl_abn_exp_5]

experiments = {}

experiments.update(calculate_mean(apsl_nor_list, "nor"))
experiments.update(calculate_mean(apsl_abn_list, "abn"))

# 生成列标签
column_labels = [f"test{i}" for i in range(1, len(apsl_nor_list) + 1)] + ["mean"]

experiments_prediction_score_df = pd.DataFrame(experiments).T
experiments_prediction_score_df.columns = column_labels
print(experiments_prediction_score_df)

# 保存为CSV文件
save_path_experiments_prediction_score_df = os.path.join(data_folder_model, 'experiments_prediction_score_df.csv')
experiments_prediction_score_df.to_csv(save_path_experiments_prediction_score_df, index=False)


In [ ]:
# 检查文件是否成功保存
if os.path.exists(save_path_experiments_prediction_score_df):
    print(f"CSV file saved successfully at: {save_path_experiments_prediction_score_df}")
else:
    print(f"Failed to save CSV file at: {save_path_experiments_prediction_score_df}")

## Save / Load Model


In [ ]:
# save model
save_model(better_model, os.path.join(data_folder_model, 'better_model'))

In [ ]:
# # load model
# loaded_from_disk = load_model('my_first_model')
# loaded_from_disk

## Save / Load Experiment


In [ ]:
# # save experiment
# save_experiment('my_experiment')

In [ ]:
# # load experiment from disk
# exp_from_disk = load_experiment('my_experiment', data=new_train_data, test_data=new_test_data,)